<a href="https://colab.research.google.com/github/pshashiraj07/shashiraj/blob/pshashiraj07-patch-1/RAG_BOT_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers faiss-cpu transformers accelerate


In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss
import numpy as np
import torch


In [ ]:
documents = [
    "Our refund policy allows customers to return products within 30 days.",
    "Business hours are Monday to Friday, 9 AM to 5 PM.",
    "We offer enterprise support plans with 24/7 service.",
    "Shipping is free on orders above $100.",
    "Technical support can be reached via email or phone."
]

In [ ]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(documents)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
embedding_dim = doc_embeddings[0].shape[0]
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(doc_embeddings))


In [ ]:
model_id = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

FalconForCausalLM(
  (transformer): FalconModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x FalconDecoderLayer(
        (self_attention): FalconAttention(
          (query_key_value): FalconLinear(in_features=4544, out_features=4672, bias=False)
          (dense): FalconLinear(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): FalconRotaryEmbedding()
        )
        (mlp): FalconMLP(
          (dense_h_to_4h): FalconLinear(in_features=4544, out_features=18176, bias=False)
          (act): GELUActivation()
          (dense_4h_to_h): FalconLinear(in_features=18176, out_features=4544, bias=False)
        )
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
    (rotary_emb): FalconRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4544, out_features=6

In [ ]:
def answer_question(question, top_k=3):
    q_embedding = embedder.encode([question])
    distances, indices = index.search(np.array(q_embedding), top_k)
    retrieved_docs = [documents[idx] for idx in indices[0]]

    context = "\n".join(retrieved_docs)
    prompt = f"Context:\n{context}\n\nQuestion: {question}\nAnswer:"

    device = "cuda" if torch.cuda.is_available() else "cpu"
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=100, do_sample=True)

    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    return answer.split("Answer:")[-1].strip()


In [ ]:
question = "What is your refund policy?"
print("Answer:", answer_question(question))


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer: - We're here to help. We'll arrange the necessary to process a refund to your PayPal account.
- It's also important to check if the product qualifies for a return by reading our Warranty Agreement. You can also read the full text of our refund policy by clicking on the link below. This policy will apply on your 2X warranty purchase to your warranty policy that requires all warranties to go to the manufacturer or third-party vendor.
- We accept the return
